<div id="singlestore-header" style="display: flex; background-color: rgba(209, 153, 255, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/vector-circle.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Launch Open-Source Apps with LangChain</h1>
    </div>
</div>

In [1]:
%%writefile requirements.txt
langchain==0.0.339
langchain-community==0.0.8
openai==1.3.3
matplotlib==3.8.2
pdf2image==1.17.0
pdfminer==20191125
pdfminer.six==20221105
pillow_heif==0.13.1
tabulate==0.9.0
tiktoken==0.5.1
unstructured==0.11.0
opencv-contrib-python-headless==4.8.1.78
unstructured.pytesseract==0.3.12
unstructured.inference==0.7.15

Writing requirements.txt


In [2]:
%conda install -y --quiet poppler==23.12.0 tesseract==5.3.3

In [3]:
%pip install -r requirements.txt --quiet

In [4]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [5]:
from langchain.document_loaders import OnlinePDFLoader

loader = OnlinePDFLoader("http://leavcom.com/pdf/DBpdf.pdf")

data = loader.load()

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

print (f"You have {len(data)} document(s) in your data")
print (f"There are {len(data[0].page_content)} characters in your document")

You have 1 document(s) in your data
There are 13040 characters in your document


In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

print (f"You have {len(texts)} pages")

You have 7 pages


In [8]:
%%sql
DROP DATABASE IF EXISTS pdf_db;
CREATE DATABASE IF NOT EXISTS pdf_db;

1 rows affected.

++
||
++
++

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Make sure to select the <tt>pdf_db</tt> database from the drop-down menu at the top of this notebook. It updates the <tt>connection_url</tt> to connect to that database.</p>
    </div>
</div>

In [9]:
%%sql
DROP TABLE IF EXISTS pdf_docs1;
CREATE TABLE IF NOT EXISTS pdf_docs1 (
    id INT PRIMARY KEY,
    content TEXT CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci,
    vector BLOB
);

++
||
++
++

In [10]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key: ········


In [11]:
import json
import sqlalchemy as sa
from langchain.embeddings import OpenAIEmbeddings
from singlestoredb import create_engine

conn = create_engine().connect()

embedder = OpenAIEmbeddings()

# Fetch all embeddings in one call
embeddings = embedder.embed_documents([doc.page_content for doc in texts])

# Build query parameters
params = []
for i, (text_content, embedding) in enumerate(zip(texts, embeddings)):
    params.append(dict(id=i+1, content=text_content, vector=json.dumps(embedding)))

stmt = sa.text("""
    INSERT INTO pdf_docs1 (
        id,
        content,
        vector
    )
    VALUES (
        :id,
        :content,
        JSON_ARRAY_PACK_F32(:vector)
    )
""")

conn.execute(stmt, params)

In [12]:
%%sql
SELECT JSON_ARRAY_UNPACK_F32(vector) as vector
FROM pdf_docs1
LIMIT 1;

1 rows affected.

vector


In [13]:
query_text = "Will object-oriented databases be commercially successful?"

query_embedding = embedder.embed_documents([query_text])[0]

stmt = sa.text("""
    SELECT
        content,
        DOT_PRODUCT_F32(JSON_ARRAY_PACK_F32(:embedding), vector) AS score
    FROM pdf_docs1
    ORDER BY score DESC
    LIMIT 1
""")

results = conn.execute(stmt, dict(embedding=json.dumps(query_embedding)))

for row in results:
    print(row[0])

page_content='are gaining in popularity and are ex- pected to outsell even relational data- bases by 2003. And OO databases (see the “OO Database Orientation” sidebar) are still minor players with solid but strictly niche markets. Sales of relational databases have grown considerably faster than the sales of OO databases, and annual worldwide RDBMS revenues are now about 50 times larger.\n\nRick Cattell, distinguished engineer at Sun Microsystems, indicated, “Object- oriented databases are doing just ﬁne, and the news of their demise is highly exag- gerated. While their market [share] isn’t as big, they continue to be used in areas like CAD (computer-aided design) and telecommunications, where RDBMSs are not well suited.”\n\nHowever, said Michael Stonebraker, chief technology ofﬁcer at Informix and an ORDBMS proponent and pioneer, “ODBMSs occupy a small niche market that has no broad appeal. The technology is in semi-rigor mortis, and ORDBMSs will corner the market within ﬁve years.”\n

In [14]:
import openai

client = openai.OpenAI()

prompt = f"The user asked: {query_text}. The most similar text from the document is: {row[0]}"

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(response.choices[0].message.content)

Based on the information provided in the document, it seems that object-oriented databases are not expected to be commercially as successful as relational databases. While they have niche markets such as in CAD and telecommunications, they are still considered minor players in the overall database market. Sales of relational databases have been growing considerably faster than sales of object-oriented databases, with a significant difference in revenues.

The document also mentions differing opinions from industry experts, with some like Rick Cattell stating that object-oriented databases are still relevant and utilized in specific areas, while others like Michael Stonebraker believe that object-relational databases will dominate the market in the coming years.

In terms of market forecasts, IDC data suggests that relational databases will continue to outperform object-oriented databases in terms of sales revenue, with the latter having slower growth rates. Object-oriented databases ar

## Clean up

In [15]:
%%sql
DROP DATABASE IF EXISTS pdf_db

1 rows affected.

++
||
++
++

<div id="singlestore-footer" style="background-color: rgba(194, 193, 199, 0.25); height:2px; margin-bottom:10px"></div>
<div><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-logo-grey.png" style="padding: 0px; margin: 0px; height: 24px"/></div>